In [1]:
spark

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 60, 6, Finished, Available)

In [2]:
# Set up data configuration
blob_account_name = "marckvnonprodblob"
blob_container_name = "bigdata"
# read only
blob_sas_token = "?sv=2021-10-04&st=2023-10-04T01%3A42%3A59Z&se=2024-01-02T02%3A42%3A00Z&sr=c&sp=rlf&sig=w3CH9MbCOpwO7DtHlrahc7AlRPxSZZb8MOgS6TaXLzI%3D"

wasbs_base_url = (
    f"wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/"
)
spark.conf.set(
    f"fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net",
    blob_sas_token,
)


StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 60, 7, Finished, Available)

In [3]:
# Read the whole Reddit data

comments_path = "reddit-parquet/comments/"
submissions_path = "reddit-parquet/submissions/"

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 60, 8, Finished, Available)

In [4]:
comments_df = spark.read.parquet(f"{wasbs_base_url}{comments_path}")
submissions_df = spark.read.parquet(f"{wasbs_base_url}{submissions_path}")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 60, 9, Finished, Available)

In [5]:
submissions_df.rdd.getNumPartitions()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 60, 10, Finished, Available)

3080

In [6]:
from pyspark.sql.functions import col, asc,desc
submissions_filtered = submissions_df.filter(col("subreddit").isin(["Minecraft", "Eldenring","Genshin_Impact","pokemongo","RocketLeagueExchange","VALORANT","DestinyTheGame","CallOfDutyMobile","FIFA","pokemon"]))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 60, 11, Finished, Available)

In [7]:
# display a subset of columns
submissions_filtered.select("subreddit", "author", "title", "selftext", "num_comments", "score").show()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 60, 12, Finished, Available)

+--------------------+--------------------+--------------------+--------------------+------------+-----+
|           subreddit|              author|               title|            selftext|num_comments|score|
+--------------------+--------------------+--------------------+--------------------+------------+-----+
|                FIFA|       Robert-Fulton|if you every get ...|                    |          12|    0|
|           Minecraft|          Dogeisagod|     Texting a house|So I’m building a...|           3|    1|
|            VALORANT|     Status_Bet_8034|       Killjoy Mains|Got bored and so ...|           2|    2|
|           Minecraft|         Age_Natural|Trident skeleton ...|                    |           3|    3|
|             pokemon|     OneWhoGetsBread|the PokemonTogeth...|So several days a...|           0|    6|
|            VALORANT|  MajesticDoughnut10|Weekly's not addi...|Since the new wee...|           1|    3|
|                FIFA|           [deleted]|should i sel

In [12]:
df_use = submissions_filtered.select("subreddit", "author", "title", "selftext", "num_comments", "score").cache()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 60, 17, Finished, Available)

In [22]:
df_use.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 60, 27, Finished, Available)

+---------+---------------+--------------------+--------------------+------------+-----+
|subreddit|         author|               title|            selftext|num_comments|score|
+---------+---------------+--------------------+--------------------+------------+-----+
|     FIFA|  Robert-Fulton|if you every get ...|                    |          12|    0|
|Minecraft|     Dogeisagod|     Texting a house|So I’m building a...|           3|    1|
| VALORANT|Status_Bet_8034|       Killjoy Mains|Got bored and so ...|           2|    2|
|Minecraft|    Age_Natural|Trident skeleton ...|                    |           3|    3|
|  pokemon|OneWhoGetsBread|the PokemonTogeth...|So several days a...|           0|    6|
+---------+---------------+--------------------+--------------------+------------+-----+
only showing top 5 rows



In [20]:
from pyspark.sql.functions import col

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 60, 25, Finished, Available)

In [31]:
from pyspark.sql.functions import (count,sum)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 60, 36, Finished, Available)

In [33]:
# Group by 'author' and perform the required aggregations
result_df = df_use.groupBy(col("author")) \
    .agg(
        col("author"),
        count("author").alias("num_posts"),
        sum("num_comments").alias("total_num_comments"),
        sum("score").alias("total_score")
    )

# Show the result DataFrame
result_df.show()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 60, 38, Finished, Available)

+--------------------+--------------------+---------+------------------+-----------+
|              author|              author|num_posts|total_num_comments|total_score|
+--------------------+--------------------+---------+------------------+-----------+
|            TheSkoko|            TheSkoko|        1|                14|          0|
|  TheGreatDarkPriest|  TheGreatDarkPriest|       39|               353|       4479|
|          alastrasix|          alastrasix|        1|                 2|          1|
|         AzureNebula|         AzureNebula|        1|                 0|          1|
|Aggravating_Trade407|Aggravating_Trade407|        1|                 0|          1|
|        Solid_Mix007|        Solid_Mix007|        4|                 7|         21|
|    retardsc00terkid|    retardsc00terkid|       18|               600|       2917|
|            sringari|            sringari|        1|                 2|          1|
|      Pezzariasimfan|      Pezzariasimfan|        1|            

In [34]:
# save the data
df_save_final=result_df.toPandas()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 60, 39, Finished, Available)

In [35]:
import os
CSV_DIR = os.path.join("Users/zx111/fall-2023-reddit-project-team-11/data", "csv")
df_save_final.to_csv(f"{CSV_DIR}/User_acitive.csv", index=False)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 60, 40, Finished, Available)